In [20]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from src.models.modules import *
from src.models.loss import L1_epsilon_lambda
from dataclasses import dataclass
import torch

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    dim_hidden: int = 128
    num_inds: int = 64
    num_heads: int = 8
    ln: bool = False

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.epsilon = None
        self.lambdaa = None
        self.proj1 = nn.Linear(config.dim_input, config.dim_hidden)
        self.proj2 = nn.Linear(config.dim_context, config.dim_hidden)
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln)
        self.enc = nn.Sequential(
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.SiLU(),
            PMA(config.dim_hidden, config.num_heads, config.num_outputs, ln=config.ln),
            nn.SiLU(),
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        x = x.expand(-1, context.shape[1], -1)  # [batch_size, context_size, dim_input]
        x = self.proj1(x)                       # [batch_size, context_size, dim_hidden]
        context = self.proj2(context)           # [batch_size, context_size, dim_hidden]
        y = self.cross(x, context)              # [batch_size, context_size, dim_hidden]
        y = self.enc(y)                         # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_epsilon_lambda(y, labels, self.epsilon, self.lambdaa, self.config.delta)
        return {'loss': loss, 'logits': y}

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [22]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [23]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_12_30_enc_mab_more_sab'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_12_30_enc_mab_more_sab-2024-12-30-12-29-17


In [24]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

curriculum_schedule = [
    {"epochs": 2, "epsilon": 0.02,   "lambda": 0.0,  'learning_rate': 5e-5},
    {"epochs": 2, "epsilon": 0.0075, "lambda": 0.15, 'learning_rate': 4e-5},
    {"epochs": 2, "epsilon": 0.004,  "lambda": 0.3,  'learning_rate': 3e-5},
    {"epochs": 2, "epsilon": 0.002,  "lambda": 0.4,  'learning_rate': 2e-5},
    {"epochs": 2, "epsilon": 0.0,    "lambda": 0.5,  'learning_rate': 1e-5}
]

In [25]:
from src.visualization.generate_mesh import generate_meshes
from src.data.load_data import get_data_dir

obj_dir = get_data_dir() / 'intermediate'
format_string_base = "{name}-" + current_date + "-curriculum-"

for i, stage in enumerate(curriculum_schedule):
    model.epsilon = stage['epsilon']
    model.lambdaa = stage['lambda']
    trainer.args.num_train_epochs = stage['epochs']
    trainer.args.learning_rate = stage['learning_rate']
    trainer.train()
    format_string = format_string_base + str(i) + ".obj"
    generate_meshes(model, obj_dir, result_dir, format_string, device,
        batch_size, resolution=100, context_size=200)
train_dataset.close()
val_dataset.close()

  0%|          | 0/18000 [00:00<?, ?it/s]

{'loss': 0.0167, 'grad_norm': 0.0, 'learning_rate': 4.997222222222223e-05, 'epoch': 0.0}
{'loss': 0.0041, 'grad_norm': 0.0, 'learning_rate': 4.994444444444445e-05, 'epoch': 0.0}
{'loss': 0.0048, 'grad_norm': 0.03555909916758537, 'learning_rate': 4.991666666666667e-05, 'epoch': 0.0}
{'loss': 0.0018, 'grad_norm': 0.0708727166056633, 'learning_rate': 4.9888888888888894e-05, 'epoch': 0.0}
{'loss': 0.0028, 'grad_norm': 0.03550346940755844, 'learning_rate': 4.986111111111111e-05, 'epoch': 0.01}
{'loss': 0.0022, 'grad_norm': 0.14221516251564026, 'learning_rate': 4.9833333333333336e-05, 'epoch': 0.01}
{'loss': 0.0024, 'grad_norm': 0.03553096950054169, 'learning_rate': 4.9805555555555554e-05, 'epoch': 0.01}
{'loss': 0.0049, 'grad_norm': 0.03648318722844124, 'learning_rate': 4.977777777777778e-05, 'epoch': 0.01}
{'loss': 0.0022, 'grad_norm': 0.03555668517947197, 'learning_rate': 4.975e-05, 'epoch': 0.01}
{'loss': 0.0034, 'grad_norm': 0.10669770836830139, 'learning_rate': 4.972222222222223e-05, '

  0%|          | 0/100 [00:00<?, ?it/s]

{'eval_loss': 0.0010872769635170698, 'eval_runtime': 3.0211, 'eval_samples_per_second': 1324.041, 'eval_steps_per_second': 33.101, 'epoch': 1.0}
{'loss': 0.0009, 'grad_norm': 0.0498361773788929, 'learning_rate': 2.4972222222222226e-05, 'epoch': 1.0}
{'loss': 0.0009, 'grad_norm': 0.0958147868514061, 'learning_rate': 2.4944444444444447e-05, 'epoch': 1.0}
{'loss': 0.0011, 'grad_norm': 0.06723348796367645, 'learning_rate': 2.4916666666666668e-05, 'epoch': 1.0}
{'loss': 0.0004, 'grad_norm': 0.08657592535018921, 'learning_rate': 2.488888888888889e-05, 'epoch': 1.0}
{'loss': 0.0003, 'grad_norm': 0.0, 'learning_rate': 2.4861111111111114e-05, 'epoch': 1.01}
{'loss': 0.0007, 'grad_norm': 0.14940032362937927, 'learning_rate': 2.4833333333333335e-05, 'epoch': 1.01}
{'loss': 0.0003, 'grad_norm': 0.0, 'learning_rate': 2.4805555555555556e-05, 'epoch': 1.01}
{'loss': 0.0012, 'grad_norm': 0.16186417639255524, 'learning_rate': 2.477777777777778e-05, 'epoch': 1.01}
{'loss': 0.0014, 'grad_norm': 0.1415395

  0%|          | 0/100 [00:00<?, ?it/s]

...

Processing models:   0%|          | 0/1 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

  0%|          | 0/18000 [00:00<?, ?it/s]

{'loss': 0.0043, 'grad_norm': 0.6617646217346191, 'learning_rate': 4.997222222222223e-05, 'epoch': 0.0}
{'loss': 0.0027, 'grad_norm': 0.5280318856239319, 'learning_rate': 4.994444444444445e-05, 'epoch': 0.0}
{'loss': 0.0021, 'grad_norm': 0.18025007843971252, 'learning_rate': 4.991666666666667e-05, 'epoch': 0.0}
{'loss': 0.0013, 'grad_norm': 0.12038377672433853, 'learning_rate': 4.9888888888888894e-05, 'epoch': 0.0}
{'loss': 0.0015, 'grad_norm': 0.16491831839084625, 'learning_rate': 4.986111111111111e-05, 'epoch': 0.01}
{'loss': 0.0008, 'grad_norm': 0.3398650288581848, 'learning_rate': 4.9833333333333336e-05, 'epoch': 0.01}
{'loss': 0.002, 'grad_norm': 0.19893500208854675, 'learning_rate': 4.9805555555555554e-05, 'epoch': 0.01}
{'loss': 0.003, 'grad_norm': 0.060048557817935944, 'learning_rate': 4.977777777777778e-05, 'epoch': 0.01}
{'loss': 0.0018, 'grad_norm': 0.09617598354816437, 'learning_rate': 4.975e-05, 'epoch': 0.01}
{'loss': 0.0013, 'grad_norm': 0.07938547432422638, 'learning_ra

KeyboardInterrupt: 

In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)